# Projet méthodes pour la science des données

# I. Chargement des données

In [2]:
import pandas as pd

df=pd.read_csv('Dataset/claimskg_result.csv')

# II. Informations sur le dataframe

In [3]:
shape=df.shape
print("Nombre d'enregistrements : ")
print(shape[0])
print("Nombre de colonnes : ")
print(shape[1])
print("Informations sur les colonnes")
df.info()

Nombre d'enregistrements : 
10000
Nombre de colonnes : 
14
Informations sur les colonnes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      10000 non-null  object
 1   text                    10000 non-null  object
 2   date                    10000 non-null  object
 3   truthRating             10000 non-null  int64 
 4   ratingName              10000 non-null  object
 5   author                  10000 non-null  object
 6   headline                9882 non-null   object
 7   named_entities_claim    9864 non-null   object
 8   named_entities_article  6497 non-null   object
 9   keywords                8691 non-null   object
 10  source                  10000 non-null  object
 11  sourceURL               10000 non-null  object
 12  link                    10000 non-null  object
 13  language          

On observe qu'il n'existe qu'un seul champs dont le type des valeurs est numérique : truthRating qui indique si l'affirmation est vraie, fausse ou un mélange de vraies et de fausses informations. Les autres types correspondent tous à des chaîne de caractères.

# III. Pré-traitements

## III.a. Supression des colonnes inutiles

Parmi les colonnes du Dataframe, certaines colonnes ne sont pas nécessaire pour les tâches de classification. On peut par exemple vérifier qu'il y'a une bijection entre l'ensemble des valeurs de truthRating et l'ensemble des valeurs de ratingName.

In [4]:
g=df.groupby(['truthRating', 'ratingName'])
print(g['id'].count())

truthRating  ratingName
-1           OTHER         1761
 1           FALSE         3665
 2           MIXTURE       3247
 3           TRUE          1327
Name: id, dtype: int64


On peut donc supprimer au choix la colonne truthRating et ratingName car les valeurs seront de toute façon transformées lors de la création des features à l'étape suivante.

In [5]:
del df['ratingName']

On peut aussi supprimer la colonne language car tous les enregistrement dans le dataframe on la valeur 'English' pour cette colonne, elle n'est donc pas utile à l'apprentissage.

In [6]:
del df['language']

In [8]:
df.to_csv('Dataset/claimskg_columns_removed.csv')

## III.b. Valeurs manquantes

Ci-dessous sont présentés les nombres de valeurs manquantes par colonnes dans le dataframe.

In [14]:
def count_empty_values(df):
    columns=df.columns[df.isnull().any()].tolist()
    for column in columns:
        print(column + " : " + str(df[column].isnull().sum()))

count_empty_values(df)

headline : 118
named_entities_claim : 136
named_entities_article : 3503
keywords : 1309


On remarque que les colonnes contenant des valeurs manquantes sont toutes de type 'chaîne de caractère', on ne peut donc pas effectuer de calcul de moyennes pour remplir les informations manquantes. Il n'est pas non plus envisageables de les supprimer car les autres colonnes portent des informations utiles à l'apprentissages.

On s'intéresse maintenant aux enregistrements qui ne portent pas la classe cible des tâches d'apprentissage :

In [31]:
print("Enregistrements non FALSE, TRUE ou MIXTURE : " + str(df[df['truthRating'] == -1].count()['id']))

Enregistrements non FALSE, TRUE ou MIXTURE : 0


Comme ces enregistrements ne peuvent servir pour l'apprentissage, on peut se permettre de les supprimer.

In [30]:
df = df[df['truthRating'] != -1]

On peut constater l'impact sur les valeurs manquantes des quatres colonnes contenant des valeurs nulles :

In [32]:
count_empty_values(df)

named_entities_claim : 11
named_entities_article : 2966
keywords : 464
